In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from src.embeddings import get_embedding_dictionnary, get_embeddings_and_word_index, sequences_to_index
from src.dataset import load_dataset, load_datasets_and_vocab_pipeline

embeddings_path = '../sentiment_analysis/data/embeddings/datastories.twitter.300d.txt'

Using TensorFlow backend.


In [3]:
(X_train, y_train), X_test, (vocab, max_len) = load_datasets_and_vocab_pipeline()

Reading twitter - 1grams ...
Reading twitter - 2grams ...
Reading twitter - 1grams ...


Building vocabulary...: 100%|██████████| 32915/32915 [00:00<00:00, 125894.72it/s]


In [4]:
emb_matrix, word_index = get_embeddings_and_word_index(embeddings_path, vocab)

KeyboardInterrupt: 

In [ ]:
X_train = sequences_to_index(X_train, word_index, max_len)
X_test = sequences_to_index(X_test, word_index, max_len)